In [ ]:
# default_exp layers

In [ ]:
#export
from sensorai.imports import *
from sensorai.tf_imports import *

In [ ]:
#hide
from nbdev.showdoc import *

# Layers
> Custom sensorai layers

## 1% Better Everyday
- [ ] [Effective TensorFlow 2 | Use tf.config.experimental_run_functions_eagerly() when debugging](https://www.tensorflow.org/guide/effective_tf2#use_tfconfigexperimental_run_functions_eagerly_when_debugging)

## Pooling layers

In [ ]:
#export
class AdaptiveConcatPool1d(keras.layers.Layer):
    def __init__(self, size=None):
        super(AdaptiveConcatPool1d,self).__init__()
        self.size = size or 1
        self.ap = tfa.layers.AdaptiveAveragePooling1D(self.size)
        self.mp = tfa.layers.AdaptiveMaxPooling1D(self.size)
        
    def call(self, x):
        return tf.concat([self.mp(x), self.ap(x)], -1)

In [ ]:
#export
class AdaptiveConcatPool2d(keras.layers.Layer):
    def __init__(self, size=None):
        super(AdaptiveConcatPool2d,self).__init__()
        self.size = size or 1
        self.ap = tfa.layers.AdaptiveAveragePooling2D(self.size)
        self.mp = tfa.layers.AdaptiveMaxPooling2D(self.size)
        
    def call(self, x):
        return tf.concat([self.mp(x), self.ap(x)], -1)

If the input is (**BS** x **W** x **H** x **CH**), the output will be (**BS** x **1** x **1** x **2*CH**) if no size is passed or (**BS** x *size* x *size* x **2*CH**)

In [ ]:
test = AdaptiveConcatPool2d()
x = tf.random.normal((10,4,4,5))
test_eq(test(x).shape, [10,1,1,10])

max1 = tf.reduce_max(x, axis=1, keepdims=True)
maxp = tf.reduce_max(max1, axis=2, keepdims=True)
test_eq(test(x)[...,:5], maxp)
test_eq(test(x)[...,5:], tf.reduce_mean(x, axis=[1,2], keepdims=True))

test = AdaptiveConcatPool2d(2)
test_eq(test(x).shape, [10,2,2,10])

In [ ]:
# export
class PoolType: Avg,Max,Cat = 'Avg','Max','Cat'

In [ ]:
#export
def adaptive_pool(pool_type):
    return (tfa.layers.AdaptiveAveragePooling2D if pool_type=='Avg' else
            tfa.layers.AdaptiveMaxPooling2D     if pool_type=='Max' else
            AdaptiveConcatPool2d)

In [ ]:
class PoolFlatten(keras.Sequential):
    "Combine `tfa.layers.AdaptiveAvgPool2d` and `tfk.layers.Flatten`."
    def __init__(self, pool_type=PoolType.Avg):
        super().__init__([adaptive_pool(pool_type)(1), keras.layers.Flatten()])

In [ ]:
test = PoolFlatten()
test_eq(test(x).shape, [10,5])
test_eq(test(x), tf.reduce_mean(x,axis=[1,2]))

test = PoolFlatten(PoolType.Cat)
test_eq(test(x).shape, [10,10])

## BatchNorm layers

## Off-the-shelf models